In [1]:
!pip install pysoundfile

In [0]:
# !rm -rf RAVDESS/spectograms/     # use this to clean the Data Set folder
# !rm -rf RAVDESS.zip  # use this to force re download

In [0]:
# helper to download the Data Set from Google Drive

import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


# Download the Data Set

Uncomment the `file_id` of the Data Set you want to use.




In [0]:
# RAVDESS_original.zip https://drive.google.com/open?id=1WyJsDuxJlUObBCFrNZLAXM2w4-sp0CxN
# file_id = '1WyJsDuxJlUObBCFrNZLAXM2w4-sp0CxN'

# RAVDESS_trimmed.zip https://drive.google.com/open?id=1pFum_YGf2C82HJvMwt0gi8apj6PJLi0s
file_id = '1pFum_YGf2C82HJvMwt0gi8apj6PJLi0s'

# RAVDESS_enriched.zip https://drive.google.com/open?id=1LG42oQTSs6HWMLsdqhFKbADA2wi8234_
# file_id = '1LG42oQTSs6HWMLsdqhFKbADA2wi8234_'

# RAVDESS_spectogram.zip https://drive.google.com/open?id=1jDb2GDnapxx-5bhRR4rudVLxd4ajrpF-
# file_id = '1jDb2GDnapxx-5bhRR4rudVLxd4ajrpF-'

# file_id ='1EuKuKF_u-ZH8CubdGmB2x_jAswqw8GBo

destination = 'RAVDESS.zip'

if not os.path.exists(destination):
    download_file_from_google_drive(file_id, destination)

In [5]:
!ls -l

total 437256
-rw-r--r-- 1 root root 447739985 Dec  8 14:15 RAVDESS.zip
drwxr-xr-x 1 root root      4096 Nov 27 22:38 sample_data


In [6]:
!unzip -n RAVDESS.zip

Archive:  RAVDESS.zip
   creating: RAVDESS/
   creating: RAVDESS/enriched/
   creating: RAVDESS/trimmed/
  inflating: RAVDESS/trimmed/03-01-07-02-01-01-23.wav  
  inflating: RAVDESS/trimmed/03-01-08-02-01-02-01.wav  
  inflating: RAVDESS/trimmed/03-01-03-02-02-02-16.wav  
  inflating: RAVDESS/trimmed/03-01-02-02-02-02-17.wav  
  inflating: RAVDESS/trimmed/03-01-06-01-02-02-22.wav  
  inflating: RAVDESS/trimmed/03-01-08-02-01-01-20.wav  
  inflating: RAVDESS/trimmed/03-01-07-02-02-02-17.wav  
  inflating: RAVDESS/trimmed/03-01-05-02-01-02-20.wav  
  inflating: RAVDESS/trimmed/03-01-06-02-02-01-03.wav  
  inflating: RAVDESS/trimmed/03-01-07-01-01-02-14.wav  
  inflating: RAVDESS/trimmed/03-01-07-02-01-01-24.wav  
  inflating: RAVDESS/trimmed/03-01-08-02-01-01-13.wav  
  inflating: RAVDESS/trimmed/03-01-07-01-02-01-04.wav  
  inflating: RAVDESS/trimmed/03-01-03-02-01-02-09.wav  
  inflating: RAVDESS/trimmed/03-01-06-02-01-01-17.wav  
  inflating: RAVDESS/trimmed/03-01-03-02-01-02-12.wav  

In [7]:
!ls RAVDESS/ -l

total 144
drwxr-xr-x 2 root root  4096 Oct 15 19:44 enriched
drwxr-xr-x 2 root root 69632 Oct 15 20:43 original
drwxr-xr-x 2 root root  4096 Oct 15 19:44 spectograms
drwxr-xr-x 2 root root 69632 Oct 15 19:50 trimmed


In [8]:
!echo "Files in RAVDESS/original:" $(find RAVDESS/original/ -type f| wc -l)
!echo "Files in RAVDESS/trimmed:" $(find RAVDESS/trimmed/ -type f| wc -l)
!echo "Files in RAVDESS/enriched:" $(find RAVDESS/enriched/ -type f| wc -l)
!echo "Files in RAVDESS/spectograms:" $(find RAVDESS/spectograms/ -type f| wc -l)
!echo "Files in RAVDESS/split_image:" $(find RAVDESS/split_image/ -type f| wc -l)

Files in RAVDESS/original: 1440
Files in RAVDESS/trimmed: 1440
Files in RAVDESS/enriched: 0
Files in RAVDESS/spectograms: 0
find: ‘RAVDESS/split_image/’: No such file or directory
Files in RAVDESS/split_image: 0


In [0]:
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
from scipy.signal import spectrogram
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import random
import re
import gc
from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from shutil import copyfile

In [0]:
random.seed(42) # set the random seed

In [0]:
# This class handles the preparation of the RAVDESS Data Set

class RAVDESS:
    
    def __init__(self, frequency):
        # store audio file name identifier mapping helpers, see the original documentation https://zenodo.org/record/1188976#.XaYUupIzaUk
        self.frequency = frequency
        self.separator = "-"
        self.audio_only = "03"
        self.voice_channel = "01"
        self.emotions = {
            "01": "neutral", 
            "02": "calm", 
            "03": "happy", 
            "04": "sad", 
            "05": "angry", 
            "06": "fearful", 
            "07": "disgust", 
            "08": "surprised"
        }
        self.emotion_intensities = {"01": "normal", "02": "strong"}
        self.statements = {"01": "Kids_are_talking_by_the_door", "02": "Dogs_are_sitting_by_the_door"}
        self.repetitions = ["01", "02"]
        self.actors = {
            "01": "male", 
            "02": "female", 
            "03": "male", 
            "04": "female", 
            "05": "male", 
            "06": "female", 
            "07": "male", 
            "08": "female", 
            "09": "male", 
            "10": "female", 
            "11": "male", 
            "12": "female", 
            "13": "male", 
            "14": "female", 
            "15": "male", 
            "16": "female", 
            "17": "male", 
            "18": "female", 
            "19": "male", 
            "20": "female", 
            "21": "male", 
            "22": "female", 
            "23": "male",
            "24": "male",
        }
        self.shown_specs = 0

        # store helper paths
        self.data_set_path = 'RAVDESS/'
        self.original_data_path = self.data_set_path + 'original/'
        self.trimmed_data_path = self.data_set_path + 'trimmed/'
        self.enriched_data_path = self.data_set_path + 'enriched/'
        self.spectograms_data_path = self.data_set_path + 'spectograms/'
        self.split_image_data_path = self.data_set_path + 'split_image/'
        self.train_valid_path = self.data_set_path + 'train_valid/'
        self.train_path = self.train_valid_path + 'train/'
        self.valid_path = self.train_valid_path + 'valid/'
        self.test_path = self.train_valid_path + 'test/'

        # create the directories if needed
        if not os.path.exists(self.trimmed_data_path):
          os.mkdir(self.trimmed_data_path)

        if not os.path.exists(self.enriched_data_path):
          os.mkdir(self.enriched_data_path)

        if not os.path.exists(self.spectograms_data_path):
          os.mkdir(self.spectograms_data_path)

        if not os.path.exists(self.split_image_data_path):
          os.mkdir(self.split_image_data_path)

        if not os.path.exists(self.train_valid_path):
          os.mkdir(self.train_valid_path)
          os.mkdir(self.train_path)
          os.mkdir(self.valid_path)
          os.mkdir(self.test_path)

        return

    # List all audio file paths
    def get_audio_paths_common(self, directory):
        audio_files = os.listdir(directory) # list all audio files
        audio_files = [directory + audio_file_name for audio_file_name in audio_files] # append prefix path
        return audio_files

    def get_original_audio_paths(self):
        return self.get_audio_paths_common(self.original_data_path)

    def get_trimmed_audio_paths(self):
        return self.get_audio_paths_common(self.trimmed_data_path)

    def get_enriched_audio_paths(self):
        return self.get_audio_paths_common(self.enriched_data_path)
    
    def get_spectograms_paths(self):
        return self.get_audio_paths_common(self.spectograms_data_path)
    
    def get_split_image_paths(self):
        return self.get_audio_paths_common(self.split_image_data_path)

    def get_file_name_without_extension(self, file_path):
        return os.path.splitext(os.path.basename(file_path))[0]

    
    # Trim the silent data from the audio files
    def trim_audios(self):
        print('Trimming audio files - remove the silent data from the audio files')
        num_of_original_files = len(os.listdir(self.original_data_path))
        num_of_trimmed_files = len(os.listdir(self.trimmed_data_path))
        print(str(num_of_trimmed_files) + '/' + str(num_of_original_files))

        audio_paths = self.get_original_audio_paths()
        finished = 0
        for audio_path in audio_paths:
            new_trimmed_file_path = self.trimmed_data_path + os.path.basename(audio_path) # on a new path with the same file name

            # if the trimmed file doesn't exist create it
            if not os.path.exists(new_trimmed_file_path):
                audio_data, samplerate = librosa.load(audio_path, sr=self.frequency) # load and convert the audio
                audio_data_filtered, index = librosa.effects.trim(audio_data, top_db=30, frame_length=2048, hop_length=512) # trim the silent data from the audio file

                librosa.output.write_wav(new_trimmed_file_path, audio_data_filtered, samplerate) # write out the trimmed data

                finished += 1
                print(str(num_of_trimmed_files + finished) + '/' + str(num_of_original_files) + ' Trimmed: ' + audio_path) # log progress
    
    # Add noise to the audio to create more samples
    def create_audio_with_noise(self, data, number_of_noisy_audios):
        mean, std = data.mean(), data.std()
        noised_data = []

        # Add noise based on the standard deviation
        for i in range(number_of_noisy_audios):
            noise = np.random.normal(mean, std*random.uniform(0.1, 0.3), data.shape) 
            noised_data.append(data + noise)
        return noised_data

    # Save the noised audio data
    def create_save_noised_audio(self, number_of_noisy_audios):
        print('Noise audios - enrich the original ' + str(number_of_noisy_audios + 1) + ' times')

        num_of_trimmed_files = len(os.listdir(self.trimmed_data_path))
        num_of_needed_enriched_files = num_of_trimmed_files * (number_of_noisy_audios + 1)
        num_of_enriched_files = len(os.listdir(self.enriched_data_path))
        print(str(num_of_enriched_files) + '/' + str(num_of_needed_enriched_files))

        audio_paths = self.get_trimmed_audio_paths()
        finished = 0
        for audio_path in audio_paths:
            audio_data, samplerate = sf.read(audio_path) # load the audio

            new_path_base = self.enriched_data_path + self.get_file_name_without_extension(audio_path) # set directory and remove file extension from the original path
            unmodified_data_new_path = new_path_base + '-00.wav' # append a new identifier Noised data ID
            if not os.path.exists(unmodified_data_new_path):
                sf.write(unmodified_data_new_path, audio_data, samplerate) # this is the non noised trimmed audio
                finished += 1
                print(str(num_of_enriched_files + finished) + '/' + str(num_of_needed_enriched_files) + ' Enrichment: ' + unmodified_data_new_path) # log progress

            # add noise to the trimmed audio
            noised_data = self.create_audio_with_noise(audio_data, number_of_noisy_audios)

            # write out the noised audio
            for i in range(0, len(noised_data)):
                new_file_name = new_path_base + '-' + "{:02d}".format(i+1) + '.wav'
                if not os.path.exists(new_file_name):
                    sf.write(new_file_name, noised_data[i], samplerate)

                    finished += 1
                    print(str(num_of_enriched_files + finished) + '/' + str(num_of_needed_enriched_files) + ' Enrichment: ' + new_file_name) # log progress
                    
                      
    def create_spectograms(self):
        print('Create spectograms')
        num_of_trimmed_files = len(os.listdir(self.trimmed_data_path))
        num_of_spectograms = len(os.listdir(self.spectograms_data_path))
        print(str(num_of_spectograms) + '/' + str(num_of_trimmed_files))

        audio_paths = self.get_trimmed_audio_paths()
        finished = 0
        for audio_path in audio_paths:
            spectogram_path = self.spectograms_data_path + self.get_file_name_without_extension(audio_path) + '.png' # create a path with the same name and a replaced file extension
            if not os.path.exists(spectogram_path):
                audio_data, samplerate = sf.read(audio_path)
                nfft = 256
                fig,ax = plt.subplots(1)
                fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
                fig.set_size_inches(2.56*len(audio_data)/samplerate,1.28)
                spect, f, t, image = plt.specgram(audio_data, NFFT=nfft, Fs=samplerate, noverlap=nfft/2, cmap=pylab.get_cmap('Greys'))
                plt.axis('off')
                fig.savefig(spectogram_path, dpi=100)

                finished += 1
                print(str(num_of_spectograms + finished) + '/' + str(num_of_trimmed_files) + ' Created spectogram: ' + spectogram_path) # log progress

                plt.close()
                gc.collect() # this won't solve the issue, but it will crash later... https://github.com/matplotlib/matplotlib/issues/8519


    def split_image_data(self, width = 200, step = 100):
        ''' overlap is width - step'''
        print('Split image data')
        num_of_spectograms = len(os.listdir(self.spectograms_data_path))
        ls = os.listdir(ravdess.split_image_data_path)
        num_of_splitted_images = len(set([i[:i.rfind('-')] for i in ls]))
        print(str(num_of_splitted_images) + '/' + str(num_of_spectograms))

        spectogram_paths = self.get_spectograms_paths()
        finished = 0
        for spectogram_path in spectogram_paths:
            split_image_path = self.split_image_data_path + self.get_file_name_without_extension(spectogram_path) + '-0.png'
            if not os.path.exists(split_image_path):
                im = Image.open(spectogram_path)
                imgwidth, imgheight = im.size
                split_image_path = self.split_image_data_path + self.get_file_name_without_extension(spectogram_path) + '-'
                for j in range(0,imgwidth,step):
                    if j + width > imgwidth:
                        start = imgwidth - width
                    else: 
                        start = j 
                    box = (start, 0, start+width, imgheight)
                    a = im.crop(box)
                    a.save(split_image_path + str(start) + '.png')
                    if j + width > imgwidth: break
                
                finished += 1
                print(str(num_of_splitted_images + finished) + '/' + str(num_of_spectograms) + ' Create split image from: ' + split_image_path )
                
                    
    
    def name_to_data(self, path):
        numbers = re.findall(r'\d+', path) # numbers in the name of the audio file
        # spectogram = Image.open(spectogram_path) # consumes too much memory
        return [
            self.emotions[numbers[2]],
            self.emotion_intensities[numbers[3]],
            self.statements[numbers[4]],
            self.repetitions[int(numbers[5])-1],
            numbers[6], 
            self.actors[numbers[6]], 
            path
        ]
        
    
    # Creates the Data Frame for the RAVDESS
    def create_dataframe(self):
        print('Create dataframe')
        data = []
        specograms_paths = self.get_split_image_paths()
        for specograms_path in specograms_paths:
            data.append(self.name_to_data(specograms_path))

        df = pd.DataFrame(  # contruct DataFrame from data
            { 
                'emotion'           : pd.Categorical([row[0] for row in data]),
                'emotion_intensity' : pd.Categorical([row[1] for row in data]),
                'statement'         : pd.Categorical([row[2] for row in data]),
                'repetition'        : pd.Categorical([row[3] for row in data]),
                'actor'             : pd.Categorical([row[4] for row in data]),
                'actor_gender'      : pd.Categorical([row[5] for row in data]),
                'image'             : pd.Categorical([row[6] for row in data])
            })

        # one-hot encode columns
        # df = pd.get_dummies(df, columns=["emotion", "emotion_intensity", "statement", "repetition", "actor", "actor_gender"])
        return df


    def create_train_structure(self, df):
        emotion = df['emotion'].unique()
        for i in emotion: 
            if not os.path.exists(self.train_path + i):
                os.mkdir(self.train_path + i)
                os.mkdir(self.valid_path + i)
                os.mkdir(self.test_path + i)
            df_mod = df[df['emotion']==i]
            train, valid_test = train_test_split(df_mod, test_size=0.2)
            valid, test = train_test_split(valid_test, test_size=0.5)

            # train:valid:test 0.8:0.1:0.1

            for index, row in train.iterrows():
                copyfile(row['image'], self.train_path+ i +'/'+self.get_file_name_without_extension(row['image'])+'.png')
            for index, row in valid.iterrows():
                copyfile(row['image'], self.valid_path+ i +'/'+self.get_file_name_without_extension(row['image'])+'.png')
            for index, row in test.iterrows():
                copyfile(row['image'], self.test_path+ i +'/'+self.get_file_name_without_extension(row['image'])+'.png')


In [0]:
frequency = 22000
ravdess = RAVDESS(frequency)

In [13]:
# STEP 1: Trim audios
ravdess.trim_audios()

Trimming audio files - remove the silent data from the audio files
1440/1440


In [14]:
# STEP 3: Create spectograms
ravdess.create_spectograms() # might crash due to https://github.com/matplotlib/matplotlib/issues/8519

Create spectograms
0/1440
1/1440 Created spectogram: RAVDESS/spectograms/03-01-06-02-01-01-17.png
2/1440 Created spectogram: RAVDESS/spectograms/03-01-04-02-02-01-08.png
3/1440 Created spectogram: RAVDESS/spectograms/03-01-06-01-01-01-01.png
4/1440 Created spectogram: RAVDESS/spectograms/03-01-01-01-02-01-22.png
5/1440 Created spectogram: RAVDESS/spectograms/03-01-02-01-02-02-23.png
6/1440 Created spectogram: RAVDESS/spectograms/03-01-08-02-02-01-07.png
7/1440 Created spectogram: RAVDESS/spectograms/03-01-02-01-01-02-02.png
8/1440 Created spectogram: RAVDESS/spectograms/03-01-05-02-01-02-24.png
9/1440 Created spectogram: RAVDESS/spectograms/03-01-02-02-01-01-12.png
10/1440 Created spectogram: RAVDESS/spectograms/03-01-07-02-01-02-12.png
11/1440 Created spectogram: RAVDESS/spectograms/03-01-04-01-01-01-02.png
12/1440 Created spectogram: RAVDESS/spectograms/03-01-06-01-01-01-05.png
13/1440 Created spectogram: RAVDESS/spectograms/03-01-06-02-01-02-15.png
14/1440 Created spectogram: RAVDES

/usr/local/lib/python3.6/dist-packages/matplotlib/axes/_axes.py:7747: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)



32/1440 Created spectogram: RAVDESS/spectograms/03-01-06-01-01-02-16.png
33/1440 Created spectogram: RAVDESS/spectograms/03-01-01-01-01-01-09.png
34/1440 Created spectogram: RAVDESS/spectograms/03-01-03-02-01-01-16.png
35/1440 Created spectogram: RAVDESS/spectograms/03-01-05-02-01-02-04.png
36/1440 Created spectogram: RAVDESS/spectograms/03-01-03-01-01-01-11.png
37/1440 Created spectogram: RAVDESS/spectograms/03-01-04-02-01-01-01.png
38/1440 Created spectogram: RAVDESS/spectograms/03-01-05-02-01-02-11.png
39/1440 Created spectogram: RAVDESS/spectograms/03-01-02-02-01-02-10.png
40/1440 Created spectogram: RAVDESS/spectograms/03-01-08-02-02-01-18.png
41/1440 Created spectogram: RAVDESS/spectograms/03-01-04-01-01-02-12.png
42/1440 Created spectogram: RAVDESS/spectograms/03-01-01-01-02-01-15.png
43/1440 Created spectogram: RAVDESS/spectograms/03-01-04-02-01-02-10.png
44/1440 Created spectogram: RAVDESS/spectograms/03-01-03-01-02-01-15.png
45/1440 Created spectogram: RAVDESS/spectograms/03

In [15]:
# STEP 4 Split the spectograms into equal sized images
ravdess.split_image_data()

Split image data
0/1440
1/1440 Create split image from: RAVDESS/split_image/03-01-07-02-01-02-19-
2/1440 Create split image from: RAVDESS/split_image/03-01-08-02-01-02-24-
3/1440 Create split image from: RAVDESS/split_image/03-01-06-01-02-01-20-
4/1440 Create split image from: RAVDESS/split_image/03-01-08-02-01-01-21-
5/1440 Create split image from: RAVDESS/split_image/03-01-07-02-01-01-12-
6/1440 Create split image from: RAVDESS/split_image/03-01-07-01-01-02-01-
7/1440 Create split image from: RAVDESS/split_image/03-01-05-02-02-01-01-
8/1440 Create split image from: RAVDESS/split_image/03-01-04-01-01-01-14-
9/1440 Create split image from: RAVDESS/split_image/03-01-06-01-01-01-13-
10/1440 Create split image from: RAVDESS/split_image/03-01-07-01-01-01-17-
11/1440 Create split image from: RAVDESS/split_image/03-01-04-01-02-01-15-
12/1440 Create split image from: RAVDESS/split_image/03-01-07-01-01-02-02-
13/1440 Create split image from: RAVDESS/split_image/03-01-08-02-01-02-20-
14/1440 Cr

In [16]:
# STEP 5: Create dataframe
df = ravdess.create_dataframe()
df.head()

Create dataframe


,emotion,emotion_intensity,statement,repetition,actor,actor_gender,image
0,calm,strong,Dogs_are_sitting_by_the_door,01,15,male,RAVDESS/split_image/03-01-02-02-02-01-15-400.png
1,fearful,normal,Dogs_are_sitting_by_the_door,01,21,male,RAVDESS/split_image/03-01-06-01-02-01-21-383.png
2,calm,normal,Dogs_are_sitting_by_the_door,01,03,male,RAVDESS/split_image/03-01-02-01-02-01-03-0.png
3,sad,normal,Kids_are_talking_by_the_door,01,07,male,RAVDESS/split_image/03-01-04-01-01-01-07-200.png
4,disgust,strong,Kids_are_talking_by_the_door,02,10,female,RAVDESS/split_image/03-01-07-02-01-02-10-400.png


In [0]:
ravdess.create_train_structure(df)

In [18]:
!echo "Files in RAVDESS/train_valid/train/angry:" $(find RAVDESS/train_valid/train/angry -type f| wc -l)
!echo "Files in RAVDESS/train_valid/train/calm" $(find RAVDESS/train_valid/train/calm -type f| wc -l)

Files in RAVDESS/train_valid/train/angry: 680
Files in RAVDESS/train_valid/train/calm 720


# Export to Google Drive

This part is optional. It is used to export the modified Data Set to your Google Drive.

In [19]:
!zip -r RAVDESS_train.zip RAVDESS/train_valid

  adding: RAVDESS/train_valid/ (stored 0%)
  adding: RAVDESS/train_valid/test/ (stored 0%)
  adding: RAVDESS/train_valid/test/surprised/ (stored 0%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-02-01-09-300.png (deflated 1%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-01-01-13-0.png (deflated 1%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-02-02-01-04-100.png (deflated 2%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-01-02-14-0.png (deflated 2%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-02-02-05-0.png (deflated 1%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-02-02-01-100.png (deflated 1%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-01-01-06-0.png (deflated 2%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-02-02-02-11-97.png (deflated 1%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-01-02-01-08-223.png (deflated 2%)
  adding: RAVDESS/train_valid/test/surprised/03-01-08-02-01-02-02-10

In [20]:
!ls -l

total 594196
drwxr-xr-x 8 root root      4096 Dec  8 14:15 RAVDESS
-rw-r--r-- 1 root root 160702055 Dec  8 14:30 RAVDESS_train.zip
-rw-r--r-- 1 root root 447739985 Dec  8 14:15 RAVDESS.zip
drwxr-xr-x 1 root root      4096 Nov 27 22:38 sample_data


In [21]:
# Export the data to your drive

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# zip name must match the name given to zip
zip_name = 'RAVDESS_train.zip'

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': zip_name})
uploaded.SetContentFile(zip_name)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1l8pPuX0SKPx9wnIfYeP7NZ-jTsNBd4nu
